In [ ]:
import yfinance as yf
import pandas as pd
import time
import os

# Load tickers from CSV
tickers_df = pd.read_csv("C:/Users/Jrans/OneDrive/Desktop/UARK/Economic Analytics/Data/Hitters.csv")
tickers = tickers_df['Ticker'].dropna().unique().tolist()  # Ensure unique, non-null tickers

# Define batch size to avoid hitting API limits
BATCH_SIZE = 100  
SAVE_PATH = "C:/Users/Jrans/OneDrive/Desktop/UARK/Economic Analytics/Data/yfinance_monthly"
os.makedirs(SAVE_PATH, exist_ok=True)  # Create folder if not exists

# Define the columns we need
columns = [
    "Ticker", "Date", "Open", "High", "Low", "Close", "Adj Close", "Volume",
    "Market Cap", "Enterprise Value", "P/E Ratio", "P/B Ratio", "Debt-to-Equity", "Beta", "Sector"
]

# Function to fetch data with retry mechanism
def fetch_data(ticker):
    for attempt in range(3):  # Retry up to 3 times
        try:
            stock = yf.Ticker(ticker)

            # Fetch monthly price data
            hist = stock.history(period="5y", interval="1mo")
            hist = hist[["Open", "High", "Low", "Close", "Volume", "Adj Close"]].reset_index()
            hist["Ticker"] = ticker  # Add ticker column

            # Fetch fundamental data (Market Cap-based ratios)
            info = stock.info
            market_cap = info.get("marketCap")
            enterprise_value = info.get("enterpriseValue")
            pe_ratio = info.get("trailingPE")
            pb_ratio = info.get("priceToBook")
            debt_to_equity = info.get("debtToEquity")
            beta = info.get("beta")
            sector = info.get("sector")

            # Append fundamental data to each row
            data = []
            for _, row in hist.iterrows():
                data.append([
                    ticker, row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Adj Close"],
                    row["Volume"], market_cap, enterprise_value, pe_ratio, pb_ratio, debt_to_equity, beta, sector
                ])
            
            return data  # Return data if successful

        except Exception as e:
            print(f"⚠️ Error retrieving {ticker} (Attempt {attempt+1}/3): {e}")
            time.sleep(2 ** attempt)  # Exponential backoff (2s, 4s, 8s)
    
    return None  # Return None if all attempts fail

# Process tickers in batches
for i in range(0, len(tickers), BATCH_SIZE):
    batch_tickers = tickers[i:i+BATCH_SIZE]
    all_data = []

    for ticker in batch_tickers:
        ticker_data = fetch_data(ticker)
        if ticker_data:
            all_data.extend(ticker_data)
        time.sleep(1)  # Prevent hitting rate limits

    if all_data:
        # Convert to DataFrame
        batch_df = pd.DataFrame(all_data, columns=columns)

        # Save batch to CSV
        batch_filename = f"{SAVE_PATH}/yfinance_batch_{i//BATCH_SIZE + 1}.csv"
        batch_df.to_csv(batch_filename, index=False)

        print(f"✅ Batch {i//BATCH_SIZE + 1} saved: {batch_filename}")

print("🎉 All batches completed successfully!")
